In [1]:
import pandas as pd
import sys
sys.path.append("/home/LAB/zuogs/zuogs-1/text_classification")
import os 
os.environ["CUDA_VISIBLE_DEVICES"]=""

train = pd.read_csv("train.csv").sample(frac=1)
test = pd.read_csv("test.csv")

In [2]:
import matplotlib.pyplot as plt

train["length"] = train["text"].apply(lambda s: len(s.split()))
train["length"].plot()
plt.show()
train['length'].describe()

<Figure size 640x480 with 1 Axes>

count    25000.000000
mean       233.787200
std        173.733032
min         10.000000
25%        127.000000
50%        174.000000
75%        284.000000
max       2470.000000
Name: length, dtype: float64

In [3]:
embed_size = 300
maxlen = 2000
max_words = 90000

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=max_words,oov_token="something")
tokenizer.fit_on_texts(train["text"])
train_words = tokenizer.texts_to_sequences(train["text"])
test_words = tokenizer.texts_to_sequences(test["text"])
train_words = pad_sequences(train_words, maxlen=maxlen)
test_words = pad_sequences(test_words, maxlen=maxlen)

/home/LAB/zuogs/py3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
import numpy as np
embeding_file = "/home/LAB/zuogs/crawl-300d-2M.vec"
def get_coef(word, *coefs):
    return word, np.asarray(coefs, dtype=np.float32)
embeding_dict=dict(get_coef(*s.strip().split(" ")) for s in open(embeding_file))

max_words = min(max_words, len(tokenizer.word_index)) + 1
embeding_matrix = np.zeros((max_words, embed_size))
lose = 0
for word, i in tokenizer.word_index.items():
    if word not in embeding_dict:
        lose += 1
        word = "something"
    if i > max_words:
        continue
    embeding_matrix[i] = embeding_dict[word]

In [9]:
from fasttext import fasttext
import tensorflow as tf
tf.reset_default_graph()
model = fasttext(input_size=maxlen, hidden_units=10, learning_rate=1e-2 ,epoch=100, batch_size=128)
model.init_session()
model.load_embeding(embeding_matrix, embeding_matrix.shape[0])
model.define_model()

train["label"] = (train["score"]>=7).astype(int)
y_target = train[["label"]].values
y_pred = np.zeros((test.shape[0],1))
nfolds = 10

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=nfolds, shuffle=True)
for tra, val in skf.split(train_words, train["label"]):
    model.train(train_words[tra], y_target[tra], train_words[val], y_target[val])
    y_pred += model.predict(test_words)
    break
y_pred /= 1#nfolds

176it [01:21,  2.16it/s]


Epoch 1: train loss : 0.675561, val loss: 0.658355
*** New best score ***



176it [01:22,  2.13it/s]


Epoch 2: train loss : 0.642764, val loss: 0.630188
*** New best score ***



176it [01:22,  2.12it/s]


...

Epoch 98: train loss : 0.351660, val loss: 0.366100
*** New best score ***



176it [01:16,  2.30it/s]


Epoch 99: train loss : 0.351330, val loss: 0.365792
*** New best score ***



176it [01:16,  2.29it/s]


Epoch 100: train loss : 0.351006, val loss: 0.365489
*** New best score ***

INFO:tensorflow:Restoring parameters from ./best_model.hdf5


In [10]:
y_pred

array([[0.75252778],
       [0.76810031],
       [0.83750604],
       ...,
       [0.08312667],
       [0.00107624],
       [0.0950447 ]])

In [11]:
from sklearn.metrics import accuracy_score as acc
acc(test["score"]>=7, y_pred>=0.5)

0.86432